In [3]:
import pandas as pd

# Load the Excel file
xls = pd.ExcelFile("2019.xlsx")  # Update path if needed
data_df = xls.parse('data')

# Filter rows where status is Withdrawn or Rejected
filtered_df = data_df[
    data_df['Live_Order_Status'].str.contains('Withdrawn|Rejected', case=False, na=False)
]

# Select required columns
result_df = filtered_df[['Client_ID', 'Reference_No', 'Live_Order_Status']]

# Display or export the result
print(result_df)
# Optionally save to a new file
# result_df.to_excel("withdrawn_rejected_output.xlsx", index=False)

     Client_ID            Reference_No Live_Order_Status
8      2019001  2025-07-KIT-COC-SC-194          Rejected
10     2019006                  FA6519          Rejected
19     2019006                  DT4790          Rejected
21     2019006                  DT4790          Rejected
25     2019006                  DT4713          Rejected
..         ...                     ...               ...
938    2019006                     NaN         Withdrawn
948    2019006                     NaN          Rejected
950    2019006                     NaN          Rejected
962    2019006                     NaN         Withdrawn
963    2019006                     NaN          Rejected

[267 rows x 3 columns]


In [6]:
import pandas as pd

# Load the Excel file
xls = pd.ExcelFile("2019.xlsx")  # Adjust the filename if needed
data_df = xls.parse('data')

# Normalize status: strip spaces and convert to lowercase
data_df['Live_Order_Status_cleaned'] = data_df['Live_Order_Status'].astype(str).str.strip().str.lower()

# Define the keywords
excluded_keywords = [
    'under review (reviewer assigned by eic)',
    'under review (reviewer assigned by our team)',
    'wo full paper submitted',
    'revised paper uploaded',
    'revision received by author',
    'submitted',
    'paper sent back to author',
    'po paper submitted',
    'published',
    'revised paper started preparing',
    'acceptance received by author',
    'comments started posting'
]

included_keywords = ['withdrawn', 'rejected', 'client need to withdraw']

# Create sets for grouping
excluded_set = set(excluded_keywords)
included_set = set(included_keywords)

# Group by Reference_No
grouped = data_df.groupby('Reference_No')

# Filter out only those Reference_Nos which have ONLY included keywords and no excluded ones
valid_refs = []

for ref_no, group in grouped:
    statuses = set(group['Live_Order_Status_cleaned'].dropna())
    if any(ek in status for ek in excluded_set for status in statuses):
        continue  # Skip if any excluded keyword is found
    if any(ik in status for ik in included_set for status in statuses):
        valid_refs.append(ref_no)

# Final filtered result
filtered_df = data_df[data_df['Reference_No'].isin(valid_refs)][['Client_ID', 'Reference_No', 'Live_Order_Status']]
print(filtered_df)
# Optionally export
filtered_df.to_excel("only_withdrawn_or_rejected_references.xlsx", index=False)

     Client_ID Reference_No        Live_Order_Status
10     2019006       FA6519                 Rejected
12     2019006       DT4148  Client Need to Withdraw
13     2019006       SP2894  Client Need to Withdraw
14     2019006       BH2530  Client Need to Withdraw
16     2019006       SP3044  Client Need to Withdraw
..         ...          ...                      ...
937    2019006      FA19429                Withdrawn
945    2019006       DT4167  Client Need to Withdraw
946    2019006      FA21623  Client Need to Withdraw
947    2019006       DT4174  Client Need to Withdraw
972    2019006       ZK2075  Client Need to Withdraw

[442 rows x 3 columns]
